### Passo 1 - Importar Arquivos e Bibliotecas

In [22]:
#import bibliotecas
import pandas as pd
import win32com.client as win32
import pathlib


#import base de dados
emails = pd.read_excel(r'database\Emails.xlsx')
vendas = pd.read_excel(r'database\Vendas.xlsx')
lojas = pd.read_csv(r'database\Lojas.csv', encoding='latin1', sep=';')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [23]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [24]:
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
    
display(dicionario_lojas['Salvador Shopping'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
96969,81,2019-01-02,8,Cueca Xadrez,1,60,60,Salvador Shopping
96970,126,2019-01-02,8,Camisa Estampa,1,113,113,Salvador Shopping
96971,138,2019-01-02,8,Pulseira Linho,5,114,570,Salvador Shopping
96972,177,2019-01-02,8,Camisa Liso,2,105,210,Salvador Shopping
96973,194,2019-01-02,8,Camisa Gola V Linho,4,145,580,Salvador Shopping
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [25]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [26]:
#identificar se a pasta já existe

caminho_backup = pathlib.Path(r'arquivos_lojas')
arquivos_backup = caminho_backup.iterdir()


#for arquivo in arquivos_backup:
#    lista_nomes_backup.append(arquivo.name)

lista_nomes_backup = [arquivo.name for arquivo in arquivos_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
        
    #salvar dentro da pasta    
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Definição de metas

In [27]:
meta_fatu_dia = 1000
meta_fatu_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

### Passo 5 - Enviar por e-mail para o gerente

In [28]:
# apenas para testar as variaveis
print(emails.loc[emails['Loja']==loja, 'E-mail'].values[0])

attachment  = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
print(attachment)

gianniniluan1+benjamin@gmail.com
C:\Users\Pc Gamer\Desktop\hashtag\automacaoprocesso\arquivos_lojas\Shopping Barra\12_26_Shopping Barra.xlsx


### Passo 6 - Automatizar todas as lojas

In [29]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]

    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador , :]

    #faturamento
    fatu_ano = vendas_loja['Valor Final'].sum()
    #print(f'Vendemos R${fatu_ano} até o momento neste ano.')

    fatu_dia = vendas_loja_dia['Valor Final'].sum()
    #print(f'Vendemos R${fatu_dia} até o momento no dia de hoje.')

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #print(f'A loja do {loja} vendeu {qtde_produtos_ano} produtos diferentes no ano.')

    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(f'A loja do {loja} vendeu {qtde_produtos_dia} produtos diferentes hoje.')

    #ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    tick_ano = valor_venda['Valor Final'].mean()
    #print(tick_ano)

    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    tick_dia = valor_venda_dia['Valor Final'].mean()
    #print(tick_dia)
    
    #Enviar o e-mail
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja{loja}'
    
    if fatu_dia >= meta_fatu_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if fatu_ano >= meta_fatu_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'


    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'


    if tick_dia >= meta_ticketmedio_dia:
        cor_tick_dia = 'green'
    else:
        cor_tick_dia = 'red'
    if tick_ano >= meta_ticketmedio_ano:
        cor_tick_ano = 'green'
    else:
        cor_tick_ano = 'red'
    
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}.</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${fatu_dia:,.2f}</td>
        <td style="text-align: center">R${meta_fatu_dia:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produto</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${tick_dia:,.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:,.2f}</td>
        <td style="text-align: center"><font color="{cor_tick_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${fatu_ano:,.2f}</td>
        <td style="text-align: center">R${meta_fatu_ano:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produto</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${tick_ano:,.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:,.2f}</td>
        <td style="text-align: center"><font color="{cor_tick_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para ánalise.</p>
    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Giannini</p>
    '''

    # Anexos (pode colocar quantos quiser):
    attachment  = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print(f'Email da Loja {loja} enviado')

Email da Loja Iguatemi Esplanada enviado
Email da Loja Shopping Midway Mall enviado
Email da Loja Norte Shopping enviado
Email da Loja Shopping Iguatemi Fortaleza enviado
Email da Loja Shopping União de Osasco enviado
Email da Loja Shopping Center Interlagos enviado
Email da Loja Rio Mar Recife enviado
Email da Loja Salvador Shopping enviado
Email da Loja Rio Mar Shopping Fortaleza enviado
Email da Loja Shopping Center Leste Aricanduva enviado
Email da Loja Ribeirão Shopping enviado
Email da Loja Shopping Morumbi enviado
Email da Loja Parque Dom Pedro Shopping enviado
Email da Loja Bourbon Shopping SP enviado
Email da Loja Palladium Shopping Curitiba enviado
Email da Loja Passei das Águas Shopping enviado
Email da Loja Center Shopping Uberlândia enviado
Email da Loja Shopping Recife enviado
Email da Loja Shopping Vila Velha enviado
Email da Loja Shopping SP Market enviado
Email da Loja Shopping Eldorado enviado
Email da Loja Shopping Ibirapuera enviado
Email da Loja Novo Shopping Ribei

### Passo 7 - Criar ranking para diretoria

In [30]:
faturamento_lojas = vendas.groupby('Loja')[['Loja','Valor Final' ]].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_ano)
   
nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'arquivos_lojas\{}'.format(nome_arquivo))

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


In [31]:
vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'arquivos_lojas\{}'.format(nome_arquivo))

,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria

In [32]:
#Enviar o e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia.

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:,.2f}
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:,.2f}

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 0]:,.2f}
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 0]:,.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att., Giannini.
'''

attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print('Email da Diretoria enviado')

Email da Diretoria enviado
